# Search and compare RNA-seq profiles
[Click here for more info on SkyMap](./README.ipynb)

In [ ]:
import skymap_widget_imports
display(skymap_widget_imports.accordion)
display(skymap_widget_imports.widget_specie)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>


### Example analysis: Simple differential expression analysis between the thousands of human B-Cells and T-Lymphocytes expression profiles  



This notebook is a template which consist of the following steps:
1. Setting input free-text regex queries to define the classes of experimental conditions (queryLabelToRegexDict)
2. Querying the reprocessed data which consist of >400,000 expression profiles
3. Generate a fully annotated expression matrix
4. DE analysis like volcano plot, correlation heatmap and PCA. (Most plots are interactive, u can download with a simple click)

### More parameters: 

expression_metric: Those are Kallisto expression metric: "tpm","est_counts"
baseDir: if run locally, change it to mirror our path.

In [ ]:
expression_metric='tpm' #

In [ ]:
querySpecie=skymap_widget_imports.widget_specie.get_interact_value()

In [ ]:
queryStr=skymap_widget_imports.widget_query.get_interact_value()

listOfQueries=queryStr.split(',')

if len(queryStr)<5:
    raise ValueError('Please provide a query with more than 5 characters')
if len(listOfQueries)<2:
    raise ValueError('Please provide a query with more than 2 conditions')


In [ ]:
queryLabelToRegexDict=dict(zip(listOfQueries,listOfQueries))

# Data loading

### load in SRS biospecieman annotations

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np

allSRS_pickle_dir='/home/jovyan/efs/all_seq/meta_data/allSRS.with_processed_data.pickle.gz'
%time allSRS=pd.read_pickle(allSRS_pickle_dir)
allSRS.index.names=['SRS','Attrib']

### load in technical metadata

In [ ]:
sra_dump_pickle_dir='/home/jovyan/efs/all_seq/meta_data/sra_dump.fastqc.bowtie_algn.pickle'
%time technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)
technical_meta_data_df[('SRAmeta','Run')]=technical_meta_data_df.index

### load the expression matrix

Check files in baseDir directory for more species

In [ ]:
def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf
data_matrix_dir=baseDir+'/{specie}.gene_symbol.{expression_metric}'.format(specie=querySpecie,
                                            expression_metric=expression_metric)

%time rnaseqDf=loadDf(data_matrix_dir)

# Find the relevent SRS (Sample  IDs)  


In [ ]:
myL=[]
for  queryRegex in queryLabelToRegexDict.values():
    %time hitSrsS=allSRS[allSRS.str.contains(queryRegex,case=False)]
    myL.append(hitSrsS)

queryLabel='queryLabel'
mergeS=pd.concat(myL,keys=queryLabelToRegexDict.keys(),names=[queryLabel])
mergeS_noDup=mergeS.groupby(['SRS','queryLabel','Attrib']).first()
unqiueHitMask=mergeS_noDup.groupby('SRS').size()==1
unqiueHitSrs=unqiueHitMask.index[unqiueHitMask]
mergeS_noDup_unique=mergeS_noDup[mergeS_noDup.index.get_level_values('SRS').isin(unqiueHitSrs)]

Number of SRS per query class

In [ ]:
mergeS_noDup_unique.groupby(queryLabel).size()

In [ ]:
srsToClasses_all=mergeS_noDup_unique.reset_index().set_index(['SRS'])['queryLabel']

srsToClasses=srsToClasses_all

### map SRS Ids to SRR Ids

In [ ]:
m_SRAMeta=technical_meta_data_df[('SRAmeta','Sample')].isin(srsToClasses.index)
technical_meta_data_df_hit=technical_meta_data_df[m_SRAMeta]

SRAMetasrsCorrespondingQuery=srsToClasses.loc[technical_meta_data_df_hit[('SRAmeta','Sample')]].values
technical_meta_data_df_hit[('SRAmeta',queryLabel)]=SRAMetasrsCorrespondingQuery
relevantMetaColsL=[('SRAmeta',queryLabel),('SRAmeta','Study'),('SRAmeta','Sample'),('SRAmeta','Run'),('SRAmeta','ScientificName')]
technical_meta_data_df_sub=technical_meta_data_df_hit[relevantMetaColsL]
designDf=technical_meta_data_df_sub['SRAmeta']

Top species with reprocessed data

In [ ]:
designDf['ScientificName'].value_counts().head(n=5)

In [ ]:
hitSrsAllAnnotS=allSRS[allSRS.index.get_level_values('SRS').isin(mergeS.index.get_level_values('SRS'))]

In [ ]:
#designDf['BioProj_description']=BioProjAccessionToDescriptionS[designDf['BioProj'].fillna('')].fillna('').values
srsToTextS=pd.Series( (hitSrsAllAnnotS.index.get_level_values('Attrib')+': '+hitSrsAllAnnotS+' <br> '),
                     index=hitSrsAllAnnotS.index).groupby(['SRS']).sum()




In [ ]:
srsToTextS=pd.Series(data="NCBI SRA SRS:"+srsToTextS.index+' <br> '+srsToTextS.values,index=srsToTextS.index)

In [ ]:
designDf['Description']=srsToTextS[designDf.Sample].values

In [ ]:
%time designDf_specie=designDf[(designDf['ScientificName']==querySpecie)&(designDf.Run.isin(rnaseqDf.columns))]
queryDesignDf=designDf_specie

Number of samples per query class with data

In [ ]:
designDf_specie.groupby(queryLabel).size()

In [ ]:
%time hitDf=pd.DataFrame( list(map( lambda srrId: rnaseqDf[srrId],queryDesignDf.Run))).T
hitDf.columns=queryDesignDf.set_index(queryDesignDf.columns.tolist()).index

### Output:  fully annnotated matrix matrix

Example layout is listed in the cell below

In [ ]:
hitDf.head()

#  Example analysis with output matrix


In [ ]:
inputAnalyzeDf=np.log2(hitDf+1)

### Interactive PCA 
to show that samples are grouped in the higher dimensional space according to the class

In [ ]:
plotPcs=[1,2]
inPcaDf=inputAnalyzeDf.T

In [ ]:
from sklearn import decomposition
import seaborn as sns
PCA=decomposition.PCA(n_components=3)
pcaM=PCA.fit_transform((inPcaDf))
pcaDf=pd.DataFrame( data=pcaM,index=inPcaDf.index)

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()
fig = go.Figure()
for label, sub_pca_df in pcaDf.groupby('queryLabel'):
    fig.add_scatter(x=sub_pca_df[plotPcs[0]], y=sub_pca_df[plotPcs[1]], name=label,
                    hovertext=sub_pca_df.index.get_level_values('Description'),
                    mode = 'markers')
iplot(fig)

# Volcano plot with t-test

Change the params in the following cell to edit the query conditions and the genes to be annotated

In [ ]:
##PARAMS
annotateGenesList=['TRGC1','IGHM']

In [ ]:
from scipy import stats
expression_effect_size_filter=0.0 
labelA=listOfQueries[0]
labelB=listOfQueries[1]
t,p=stats.ttest_ind(inputAnalyzeDf[labelA],inputAnalyzeDf[labelB],axis=1)
effectDiff=inputAnalyzeDf[labelA].mean(axis=1)-inputAnalyzeDf[labelB].mean(axis=1)
effectLabel='expression of : "{}" - "{}"'.format(labelA, labelB)
tmpDf=pd.DataFrame({'t':t,'-log10(p)':-np.log10(p),effectLabel:effectDiff,'u':inputAnalyzeDf.mean(axis=1)},index=inputAnalyzeDf.index)
plotDf=tmpDf[tmpDf['u']>=expression_effect_size_filter].dropna()
yLabel='-log10(p)'
xLabel=effectLabel

#### Interactive scatter plot with plotly

In [ ]:
fig = go.Figure()
m=plotDf.index.isin(annotateGenesList)
fig.add_scatter( x=plotDf[xLabel][~m],y=plotDf[yLabel][~m],mode='markers',hovertext=plotDf.index.values,name='')
fig.add_scatter( x=plotDf[xLabel][m],y=plotDf[yLabel][m],mode='markers+text',text=plotDf.index.values,
                textposition='bottom center',
                name='Query annotation gene')
iplot(fig)

In [ ]:
rankS=tmpDf['-log10(p)'].fillna(1.0).sort_values().rank()/tmpDf.shape[0]

Unsurprisingly, the IGHM gene which encodes the constant region of the B-cell receptor are highly expressed in B-Cells when compared with T-Cells.


### Boxplot showing the expression level of a gene

Change the queryGene to something else

In [ ]:
queryGene='IGHM' 

In [ ]:
upRegDf=tmpDf[tmpDf[effectLabel]>0]
rankOfGene=(upRegDf['-log10(p)'].rank()/upRegDf.shape[0]).loc[queryGene]*100
print ('{} is the top {}% expressed gene in {} '.format(queryGene,rankOfGene,labelA))

In [ ]:
layout = go.Layout(
yaxis={'title':"Expression level of gene "+queryGene+' measured in log2 '+expression_metric.upper()}
)
data=[go.Box(name=myQueryLabel,y=tmpS.values)
      for myQueryLabel, tmpS in inputAnalyzeDf.loc[queryGene].groupby('queryLabel')]
fig=go.Figure(data=data,layout=layout)

iplot(fig )


Unsurprisingly, the TRGC1 gene which encodes the constant region of the T-Cell receptors are highly expressed in  T-Cells when compared with B-Cells.



In [ ]:
queryGene='TRGC1'
downRegDf=tmpDf[tmpDf[effectLabel]<0]
rankOfGene=(downRegDf['-log10(p)'].rank()/downRegDf.shape[0]).loc[queryGene]*100
print ('{} is the top {}% expressed gene in {} '.format(queryGene,rankOfGene,labelB))

### Study level condition correlation heatmap

Useful for finding outliers

In [ ]:
%matplotlib inline
myColorL=['red','blue','green','orange','pink'] ##add more colors as you see fit
sigDf=inputAnalyzeDf.groupby(level=['queryLabel','Study'],axis=1).mean()
corrDf=sigDf[sigDf.mad(axis=1)>np.log2(5)].corr()
corrDf_valid=corrDf.loc[~corrDf.isnull().all(axis=1),~corrDf.isnull().all(axis=0)]
labelToColorS=pd.Series(dict(zip(corrDf_valid.index.get_level_values('queryLabel').unique(),myColorL)))
print (labelToColorS)
colors=labelToColorS[corrDf_valid.index.get_level_values('queryLabel')].values
g=sns.clustermap(data=corrDf_valid,col_colors=colors,metric='euclidean')
g

# Caveats

### Caveat: the mode of expression from public sequencing data usually don't have same center, that's why the Volcano plot from above is skewed

You might want to do a median/mean center in your dataset

In [ ]:
inputAnalyzeDf[labelA].mean(axis=1).median(),\
inputAnalyzeDf[labelB].mean(axis=1).median()

### Caveat: Batch effect
If the first PC (PC0 in the following plot) does not seperate the  experimental condition, this is most likely due to batch effect. Sometimes plotting second and third PC like the above plot will find the seperation.

If it doesn't do the job, I recommend doing a simple z-score of the data to subtract out the covariates before going down the route of using batch correction tools like: [Combat, SVR](https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0030161) or DEseq. 

In [ ]:
plotPcs=[0,1]
sns.pairplot(data=pcaDf.reset_index()[plotPcs+[queryLabel]],hue=queryLabel,plot_kws={'alpha':0.2})

# Content page for completed results

* [Output: fully annnotated matrix matrix](#Output:--fully-annnotated-matrix-matrix)
* [Interactive PCA](#Interactive-PCA)
* [Volcano plot with t-test](#Volcano-plot-with-t-test)
* [Boxplot showing the expression level of a gene ](#Boxplot-showing-the-expression-level-of-a-gene)
* [Study level condition correlation heatmap](#Study-level-condition-correlation-heatmap)